In [ ]:
def softmax(x, print_):
    if print_==1:
        print(x)
    x = x-np.max(x)
    if print_==1:
        print(x)
    exp_x = np.exp(x)
    if print_==1:
        print('exp_x', exp_x)
        print(exp_x/np.sum(exp_x))
    return exp_x/np.sum(exp_x)

In [ ]:
def sample_multi(net_out_, temperature, speech):
    net_out = softmax(np.asarray(net_out_), 0)[0]
    out_1 = np.log(net_out)/(temperature+1e-5)
    exp_out = np.exp(out_1)
    out_2 = exp_out/np.sum(exp_out)
    out_3 = copy.deepcopy(out_2)
    out_3[out_3<0.0001] = 0
    out_4 = out_3/(np.sum(out_3)+1e-6)
    out_5 = copy.deepcopy(out_4)
    out_5[-(1+speech)] = 0
    try:
        probs = np.random.multinomial(1, out_5, 1)
    except:
        print('net_out', net_out)
        print('out_1', out_1)
        print('exp_out', exp_out)
        print('out_2', out_2)
        print('out_3', out_3)
        print(out_5)
        _ = softmax(np.asarray(net_out_), 1)[0]
    if speech==0 and (np.argmax(probs)==(len(probs)-1)):
        return 0
    return np.argmax(probs)

In [ ]:
def generate_text_multi(net, seed, LWds, file=None, temperature=0., max_char=500):

    net.eval() # Evaluation mode (e.g. disable dropout)

    print_on_file = seed
    
    speech = 0
    
    net.to('cpu')
    
    #%% Find initial state of the RNN
    with torch.no_grad():
        # Encode seed
        seed_encoded = encode_text(LWds.char_to_number, seed)
        seed_encoded = [0]*(LWds.n_char-len(seed_encoded)-1) + seed_encoded
        
        open_speech = np.sum(np.array(seed_encoded)==(len(LWds.alphabet)-2))
        close_speech = np.sum(np.array(seed_encoded)==(len(LWds.alphabet)-1))
        if (open_speech-close_speech)==-1:
            speech = 1
        
        seed_rnn = create_one_hot_matrix(seed_encoded, len(LWds.alphabet))
        seed_rnn = torch.tensor(seed_rnn).float()
        seed_rnn = seed_rnn.unsqueeze(0)
        
        net_out, state = net(seed_rnn, in_speech = speech)
        
        next_char_encoded = sample_multi(net_out, temperature, speech)
        if next_char_encoded==(len(LWds.alphabet)-2):
            speech = 1
        elif next_char_encoded==(len(LWds.alphabet)-1):
            speech = 0
        seed_encoded.append(next_char_encoded)
        seed_encoded = seed_encoded[1:]
        
        next_char = LWds.number_to_char[next_char_encoded]
        if not file:
            print(seed, end='', flush=True)
        if not file:
            print(next_char, end='', flush=True)
        else:
            print_on_file += next_char
    
    #%% Generate chapter
    new_line_count = 0
    tot_char_count = 0
    while True:
        with torch.no_grad():
        
            seed_rnn = create_one_hot_matrix(seed_encoded, len(LWds.alphabet))
            seed_rnn = torch.tensor(seed_rnn).float()
            seed_rnn = seed_rnn.unsqueeze(0)
            
            net_out, state = net(seed_rnn,
                                 state,
                                 in_speech = speech)
            
            next_char_encoded = sample_multi(net_out, temperature, speech)
            if next_char_encoded==(len(LWds.alphabet)-2):
                speech = 1
            elif next_char_encoded==(len(LWds.alphabet)-1):
                speech = 0
            
            seed_encoded.append(next_char_encoded)
            seed_encoded = seed_encoded[1:]
                
            next_char = LWds.number_to_char[next_char_encoded]
            if not file:
                print(next_char, end='', flush=True)
            else:
                print_on_file += next_char
                
            tot_char_count += 1
            if next_char == '\n':
                new_line_count += 1
            if new_line_count == 20 or tot_char_count > max_char:
                break
    if file:
        f = open(file, 'w+')
        f.write(print_on_file)
        f.close()
    net.to('cuda')
    net.train()